In [10]:
from dotenv import load_dotenv

load_dotenv()

True

# 1. RAGAS 설치 

In [1]:
# ! pip install ragas


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. 합성 테스트 데이터셋 생성 
- RAGAS: https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/

## 2.1 기본 세팅

1. 문서 로드

In [49]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')
docs = loader.load()[3:-1]

print(len(docs))

21


2. 로드된 문서의 각 metadata에 filename 속성 추가
    - filename속성은 Test Datasets 생성 프로세스에서 활용됨
    - 동일한 문서에 속한 청크를 식별하는 데 사용 됨 (source와 같은 역할)
    - source의 내용을 filename속성에 복사해주기

In [50]:
# 메타데이터 확인
docs[0].metadata

{'source': './data/SPRi AI Brief_10월호_산업동향_F.pdf',
 'file_path': './data/SPRi AI Brief_10월호_산업동향_F.pdf',
 'page': 3,
 'total_pages': 25,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13947',
 'Producer': 'Hancom PDF 1.3.0.547',
 'CreationDate': "D:20241007090546+09'00'",
 'ModDate': "D:20241007090546+09'00'",
 'PDFVersion': '1.4'}

In [51]:
# 각 Document객체의 metadata에 filename속성 추가
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

In [52]:
# filename 추가됐는지 확인
docs[0].metadata

{'source': './data/SPRi AI Brief_10월호_산업동향_F.pdf',
 'file_path': './data/SPRi AI Brief_10월호_산업동향_F.pdf',
 'page': 3,
 'total_pages': 25,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13947',
 'Producer': 'Hancom PDF 1.3.0.547',
 'CreationDate': "D:20241007090546+09'00'",
 'ModDate': "D:20241007090546+09'00'",
 'PDFVersion': '1.4',
 'filename': './data/SPRi AI Brief_10월호_산업동향_F.pdf'}

## 2.2 데이터셋 생성
- Q&A dataset 종류
    - simple: 단순 질의응답 데이터셋
    - reasioning: 추론 능력을 확인할 수 있는 데이터셋
    - multi_context: 여러 문맥을 고려하여 답 생성하는 데이터셋 
    - conditional: 조건부 질의 응답 

1. 모델 생성

In [53]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기 생성 (llm으로 데이터셋/질문 생성)
generator_llm = ChatOpenAI(model='gpt-4o-mini')

# 데이터셋 비평기 생성 (llm으로 질문이 적합한 질문인지 평가)
critic_llm = ChatOpenAI(model='gpt-4o-mini')

# 임베딩 모델 객체 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

2. 메모리 객체 생성 준비 (InMemory객체 옵션에 필요한 객체 생성)

In [54]:
# splitter 생성
splitter = RecursiveCharacterTextSplitter(
    chunk_size=550,
    chunk_overlap=150
)

# ragas와 호환되는 llm 생성
# Lanchain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 RAGAS와 호환되도록 함
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model='gpt-4o-mini'))

# 문서에서 주요 구문(KeyPhrase)를 먼저 뽑기 위한 객체 생성(ragas와 호환되는 llm 사용)
# 즉, 문서의 주요 구문에서 질문과 답변을 뽑아냄
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas와 호환되는 embeddings 객체 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore 객체 생성 (key phrase를 메모리에 올리는 역할)
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor
)

3. TestSet 생성해주는 생성기 생성

In [55]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore
)

4. 질의응답 유형별 분포 

In [56]:
# 질문 유형별 분포 설정
# 질문을 10개 생성할 경우 각각 4개, 2개, 2개, 2개씩 생성
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

5. TestDataSet 생성

In [57]:
# test dataset 객체 생성
test_dataset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True
)

embedding nodes:   0%|          | 0/158 [00:00<?, ?it/s]

[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['GPU utilization', 'AI training', 'Compute Desert', 'Compute North', 'Compute South']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI boost', 'UK job market', 'Workforce Skills Forecast', 'Techradar', 'Servicenow']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['UN AI Guidelines', 'Artificial Intelligence', '2024 Report', 'Global Standards', 'AI Policy Framework']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['GPT-4o', 'AI applications', 'o1 model', 'Chain of Thought', 'Advanced features']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['AI World', 'AI Congress 2024', 'ML and AI Model Development', 'EU AI Regulation', 'Governance']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['C2PA Trust List', 'AI content transparency', 'Deepmind SynthID', 'Generative AI', 'Content verification']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Claude 3.5 Sonnet', 'AI safety research

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 3, 'score': 2.0}
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 3, 'score': 2.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['AI regulations', 'California Legislature', 'Bill SB 1047', 'AI safety', 'Gavin Newsom']
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['AI regulations', 'California Legislature', 'Bill SB 1047', 'AI safety', 'Gavin Newsom']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['Cypher Conference 2024', 'AI World Congress', 'AI technology advancements', 'AI applications', 'Santa Clara']
[ragas.testset.evolutions.DEBUG] keyphrases in m

In [69]:
print(type(test_dataset))
print(len(test_dataset.test_data))
test_dataset.test_data[0].__annotations__

<class 'ragas.testset.generator.TestDataset'>
10


{'question': 'str',
 'contexts': 't.List[str]',
 'ground_truth': 't.Union[str, float]',
 'evolution_type': 'str',
 'metadata': 't.List[dict]'}

6. DataFrame으로 변형
    - question: 생성된 질문
    - contexts: 질문의 근거가 되는 청크
    - ground_truth: llm이 만들어낸 정답 답변
    - evolution_type: 질의 유형

In [70]:
# dataframe으로 변형
test_df = test_dataset.to_pandas()
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What do working Americans believe about the im...,[∙ 전체 응답자의 71%는 자신을 포함해 AI 발전으로 인해 실직하거나 근무 시간...,About half of working Americans believe that A...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
1,What was the outcome of the proposed Bill SB 1...,[∙ 뉴섬 주지사는 소형 AI 모델이 민감한 데이터를 다루는 위험한 작업에 사용되거...,The proposed Bill SB 1047 regarding AI restric...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
2,What is the purpose of Deepmind's SynthID in r...,[말 관련 업데이트를 진행 예정\n∙ 향후 발표될 ‘C2PA 신뢰 목록(Trust ...,The purpose of Deepmind's SynthID in relation ...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
3,What is the significance of political represen...,[n 동의 없는 딥페이크 음란물의 공유를 금지하는 이번 법안은 의회 전반의 폭넓은 ...,The answer to given question is not present in...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
4,How does AI impact student engagement in educa...,"[∙ 영국 정부는 커리큘럼 지침, 수업 계획, 익명화된 학생 평가와 같은 정부 문서...",AI impacts student engagement in education by ...,reasoning,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True


In [71]:
test_df.shape

(10, 6)

In [72]:
test_df.iloc[0]['ground_truth']

'About half of working Americans believe that AI will decrease the number of available jobs in their industry.'

7. csv 파일로 저장

In [73]:
test_df.to_csv('./data/ragas_test_dataset.csv', index=False) # df의 인덱스는 포함 안 함

# 3. 평가

1. csv 파일 로드 

In [2]:
import pandas as pd

df = pd.read_csv('./data/ragas_test_dataset.csv')
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What do working Americans believe about the im...,['∙ 전체 응답자의 71%는 자신을 포함해 AI 발전으로 인해 실직하거나 근무 시...,About half of working Americans believe that A...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
1,What was the outcome of the proposed Bill SB 1...,['∙ 뉴섬 주지사는 소형 AI 모델이 민감한 데이터를 다루는 위험한 작업에 사용되...,The proposed Bill SB 1047 regarding AI restric...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
2,What is the purpose of Deepmind's SynthID in r...,['말 관련 업데이트를 진행 예정\n∙ 향후 발표될 ‘C2PA 신뢰 목록(Trust...,The purpose of Deepmind's SynthID in relation ...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
3,What is the significance of political represen...,['n 동의 없는 딥페이크 음란물의 공유를 금지하는 이번 법안은 의회 전반의 폭넓은...,The answer to given question is not present in...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True
4,How does AI impact student engagement in educa...,"['∙ 영국 정부는 커리큘럼 지침, 수업 계획, 익명화된 학생 평가와 같은 정부 문...",AI impacts student engagement in education by ...,reasoning,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True


2. DataFrame을 Dataset 객체로 변환

In [3]:
from datasets import Dataset

# ragas에서 평가 시 Dataset클래스로 매핑이 돼있어야 됨
test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

3. df의 contexts를 list 타입으로 벼환해주기
- 겉보기엔 list처럼 보이지만 사실 str 값임 

In [4]:
# contexts 칼럼 타입 확인 
print(type(df.iloc[0]['contexts']))

<class 'str'>


In [77]:
import ast

# contexts 칼럼 데이터를 list로 변환 
def conver_to_list(example):
    contexts = ast.literal_eval(example['contexts']) # literal_eval: 문자열을 파이썬 객체로 변환해줌 
    return {'contexts': contexts}

# map
#   입력: Dataset의 각 행이 사용자 정의 함수에 입력으로 들어감
#   출력: 사용자 정의함수가 적용된 새로운 Dataset이 반환됨
test_dataset = test_dataset.map(conver_to_list) 
print(test_dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})


In [78]:
# 타입 변환된 거 확인
print(type(test_dataset[0]['contexts']))

<class 'list'>


4. 예제 프롬프트 파이프라인

In [79]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. 문서 로드
loader = PDFPlumberLoader('./data/SPRi AI Brief_10월호_산업동향_F.pdf')
docs = loader.load()[3:-1]

# 2. 문서 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=550,
    chunk_overlap=150
)
split_docs = splitter.split_documents(docs)

# 3. 임베딩 객체 생성
embeddings = OpenAIEmbeddings()

# 4. vectorstore 생성 및 저장 
vectorstore = FAISS.from_documents(
    documents=split_docs,
    embedding=embeddings
)

# 5. 검색기 생성
retriever = vectorstore.as_retriever()

# 6. 프롬프트 생성
prompt = PromptTemplate.from_template(
    """
당신은 친절한 챗봇입니다. 사용자의 질문에 대답해주세요.
사용자의 질문이 들어오면 context에 기반하여 question에 대답해주세요.

#Context:
{context}

#Question:
{question}

#Answer:
"""
)

# 7. llm 객체 생성
model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# 8. chain 생성
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

5. batch 데이터셋 생성
    - 다량의 질문을 한 번에 처리할 때 유용

In [80]:
batch_dataset = [question for question in test_dataset['question']]
batch_dataset[:5]

['What do working Americans believe about the impact of AI on job availability in their industry?',
 'What was the outcome of the proposed Bill SB 1047 regarding AI restrictions in California?',
 "What is the purpose of Deepmind's SynthID in relation to AI-generated content?",
 'What is the significance of political representation in the context of governance and decision-making?',
 'How does AI impact student engagement in education?']

6. 평가용 질문으로 답변 받아보기 
    - batch 방식 사용

In [81]:
result = chain.batch(batch_dataset)

# 답변
result

['Working Americans have mixed opinions about the impact of AI on job availability in their industry. According to a YouGov survey, 48% of American workers believe that the advancement of AI will lead to a decrease in available jobs, while 34% express concerns about job loss, reduced working hours, or decreased wages due to AI. However, 63% of respondents do not worry about job loss or changes in working conditions related to AI, and only a small percentage (2%) reported having experienced job loss or reduced hours/wages due to AI. Overall, there is a significant divide in perceptions, with some fearing negative impacts while others remain optimistic.',
 'The proposed Bill SB 1047 regarding AI restrictions in California was ultimately vetoed by Governor Gavin Newsom. Although the bill had passed the California State Assembly and aimed to impose various safety measures on AI companies, Newsom expressed concerns that the bill focused too heavily on regulating large and expensive AI model

7. 답변을 test_dataset의 answer 컬럼에 저장하기 (answer컬럼 새로 생성해줘야 함)

In [82]:
# answer 컬럼 덮어 씌우기 또는 추가
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', result)
else:
    test_dataset = test_dataset.add_column('answer', result)

In [83]:
print(type(test_dataset))
test_dataset.column_names

<class 'datasets.arrow_dataset.Dataset'>


['question',
 'contexts',
 'ground_truth',
 'evolution_type',
 'metadata',
 'episode_done',
 'answer']

## 3.1  답변 평가  

1. Context Recall 
    - 검색된 context가 LLM이 생성한 답변과 얼마나 일치하는지 측정 
        - 10개의 chunks를 뽑아냈는데 10개 모두에 답변에 필요한 내용이 담겨있는 경우 
        - 즉, ground-truth가 10개의 문서에 모두 있는 경우 성능이 좋음을 의미
    - 0~1 사이의 값
    - 값이 높을 수록 좋은 성능을 의미

2. Context Precision
    - 얼마나 관련성 있는 문서가 상위에 배치되었는가?
    - 즉, ground-truth가 포함된 문서가 얼마나 상위에 배치돼있는가를 판단 
    - 예를 들어) 답변에 필요한 문서가 3개인데 3개가 retriever로 뽑은 10개의 문서 중 상위 3위안에 들면 성능은 만점이 나옴

3. Answer Relevancy
    - 생성된 답변이 주어진 prompt에 얼마나 적절한지를 평가 
    - 즉 생성된 답변이 원래 질문의 의도를 얼마나 잘 반영하는지를 측정 
    - 질문의 임베딩과 생성된 답변의 임베딩의 코사인 유사도 측정
        - 답변이 질문과 연관성이 높으면 높은 점수를 받음 
    - 0~1사이의 값을 가지거나 코사인 유사도 특성상 수학적으로 -1~1 사이의 값을 가질 수도 있음 

4. Faithfulness
    - 생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정하는 지표 
    - 즉, 주어진 context에 얼마나 충실히 (정확히) 답변했는지 평가 

In [84]:
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

{'context_precision': 0.5000, 'faithfulness': 0.5708, 'answer_relevancy': 0.7633, 'context_recall': 0.8000}

In [85]:
print(type(result))

<class 'ragas.evaluation.Result'>


In [86]:
result_df = result.to_pandas()
result_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer,context_precision,faithfulness,answer_relevancy,context_recall
0,What do working Americans believe about the im...,[∙ 전체 응답자의 71%는 자신을 포함해 AI 발전으로 인해 실직하거나 근무 시간...,About half of working Americans believe that A...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True,Working Americans have mixed opinions about th...,1.0,0.571429,0.000000,1.0
1,What was the outcome of the proposed Bill SB 1...,[∙ 뉴섬 주지사는 소형 AI 모델이 민감한 데이터를 다루는 위험한 작업에 사용되거...,The proposed Bill SB 1047 regarding AI restric...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True,The proposed Bill SB 1047 regarding AI restric...,1.0,0.750000,0.975610,1.0
2,What is the purpose of Deepmind's SynthID in r...,[말 관련 업데이트를 진행 예정\n∙ 향후 발표될 ‘C2PA 신뢰 목록(Trust ...,The purpose of Deepmind's SynthID in relation ...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True,Deepmind's SynthID is a watermarking technolog...,1.0,0.444444,0.951854,1.0
3,What is the significance of political represen...,[n 동의 없는 딥페이크 음란물의 공유를 금지하는 이번 법안은 의회 전반의 폭넓은 ...,The answer to given question is not present in...,simple,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True,정치적 대표성은 거버넌스와 의사결정 과정에서 매우 중요한 역할을 합니다. 이는 다양...,0.0,1.000000,0.000000,1.0
4,How does AI impact student engagement in educa...,"[∙ 영국 정부는 커리큘럼 지침, 수업 계획, 익명화된 학생 평가와 같은 정부 문서...",AI impacts student engagement in education by ...,reasoning,[{'source': './data/SPRi AI Brief_10월호_산업동향_F....,True,AI can significantly enhance student engagemen...,1.0,0.692308,0.974894,0.0


In [87]:
result_df.loc[:, 'context_precision':'context_recall']
# precision(1.0이 나올 경우 ground-truth가 포함된 context의 유사도가 1순위임을 의미)
# faithfulness(사실 여부 판단)
# answer_relevancy(prompt에따라 잘 답변 했는지 정도)
# context_recall(검색된 context가 LLM이 생성한 답변(answer)과 얼마나 일치하는지)

,context_precision,faithfulness,answer_relevancy,context_recall
0,1.0,0.571429,0.000000,1.0
1,1.0,0.750000,0.975610,1.0
2,1.0,0.444444,0.951854,1.0
3,0.0,1.000000,0.000000,1.0
4,1.0,0.692308,0.974894,0.0
5,0.0,0.250000,0.923297,1.0
6,1.0,1.000000,0.961460,0.0
7,0.0,0.000000,0.980583,1.0
8,0.0,0.400000,0.967412,1.0
9,0.0,0.600000,0.897472,1.0
